In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1633202902274_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# ler dados do enenm no S3
enem = spark.read\
        .format('csv')\
        .option('header', True)\
        .option('inferSchema', True)\
        .option('delimiter', ';')\
        .load('s3://bucket-igti-roncato/raw-data/enem/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
enem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO_MUNICIPIO_NASCIMENTO: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MU

In [5]:
enem.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5095270

In [7]:
# criando df com algumas colunas
df = enem.select('NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT')        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5095270

In [10]:
from pyspark.sql.functions import max, count, mean, col, min

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# agrupando algumas informalções do df
df\
.groupby('TP_SEXO')\
.agg(
    mean(col('NU_IDADE')).alias('MED_IDADE'),
    count(col('TP_SEXO')).alias('CONTAGEM'),
    max(col('NU_NOTA_MT')).alias('NOTA_MAXIMA'),
    min(col('NU_NOTA_MT')).alias('NOTA_MINIMA')
)\
.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------+-----------+-----------+
|TP_SEXO|         MED_IDADE|CONTAGEM|NOTA_MAXIMA|NOTA_MINIMA|
+-------+------------------+--------+-----------+-----------+
|      M|22.179410259492027| 2063448|      985.5|        0.0|
|      F|22.025187142228095| 3031822|      985.0|        0.0|
+-------+------------------+--------+-----------+-----------+

In [ ]:
# salvar o dado em parquet no S3
enem\
    .write\
    .mode('overwrite')\
    .format('parquet')\
    .partitionBy('year')\
    .save('s3://bucket-igti-roncato/staging/enem/')